In [ ]:
# Imports
import pandas as pd
import string 
import numpy as np
import csv
import math
from torch import nn
import torch
from torch.utils.data import DataLoader,TensorDataset
from torchvision import transforms
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

In [ ]:
#TODO: implement!

from google.colab import files
uploaded = files.upload()

Saving hindi_hatespeech.tsv to hindi_hatespeech.tsv
Saving model to model
Saving stopword.txt to stopword.txt


In [ ]:
data = pd.read_csv('hindi_hatespeech.tsv', sep='\t')
# data = data.head(100)
stop_word_list = pd.read_csv('stopword.txt', sep='\s+', header=None)
stop_word_list = stop_word_list[0].tolist()
data['text'] = data['text'].str.replace('[{}]'.format(string.punctuation), '')
data['text'] = data['text'].str.replace('[{}]'.format('।'), '')
# data['text'] = data['text'].str.replace('[{}]'.format('\n'), '')
data['text'] = data['text'].str.lower()
data['text'] = data['text'].apply(lambda x: ' '.join([item for item in x.split() if item not in (stop_word_list)]))
data['task_1'] = data['task_1'].map({'HOF': 1, 'NOT': 0})

In [ ]:
#TODO: implement!
V = list(data['text'].str.split(' ', expand=True).stack().unique())
print(len(V))
def word_to_one_hot(word, vocab):
  return vocab[word]

20241


In [ ]:
df = pd.DataFrame(list(zip(V)), columns=['Vocalbulary'])
one_hot_vocabulary = pd.get_dummies(df.Vocalbulary)

In [ ]:
for each in range(len(data['text'])):
  if len(data['text'][each].split(" "))<= 25:
    for i in range(25 - len(data['text'][each].split(" "))):
      data['text'][each] = data['text'][each] + " 0"
  elif len(data['text'][each].split(" "))> 25:
    x = data['text'][each].split(" ")[:25]
    data['text'][each] = " ".join(x)
  else:
    pass

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [ ]:
# Create model 

class Word2Vec(nn.Module):
  def __init__(self, length, embedding_size):
    super().__init__()
    # print("First.")
    self.input_layer = nn.Linear(in_features=length, out_features=embedding_size, bias=False)
    # print("Second.")
    self.output = nn.Linear(in_features=embedding_size, out_features=length, bias=False)

  def forward(self, one_hot):
    x=self.input_layer(one_hot)
    x2 = self.output(x)
    out = F.log_softmax(x2)
    return out

# torch.cuda.empty_cache()
# model = Word2Vec(len(word_probability_dictionary), embedding_size)
# if torch.cuda.is_available():
#     model.cuda()


In [ ]:
# Previous model load.
model = Word2Vec(20150, 300)
# model.load_state_dict(torch.load("/content/model"))
model.load_state_dict(torch.load("/content/model", map_location=torch.device('cpu')))
  

<All keys matched successfully>

In [ ]:
print(model.input_layer.weight)

Parameter containing:
tensor([[-0.4633, -1.0554, -1.0619,  ...,  0.5338, -0.5103,  0.5778],
        [-0.4655, -1.0529, -1.0604,  ...,  0.5328,  0.5096,  0.5848],
        [-0.4710, -1.0515, -1.0609,  ..., -0.5349, -0.5120, -0.5819],
        ...,
        [ 0.4599, -1.0524, -1.0631,  ...,  0.5267,  0.5159,  0.5798],
        [-0.4655,  1.0513,  1.0555,  ...,  0.5314,  0.5080,  0.5894],
        [-0.4622,  1.0473, -1.0519,  ..., -0.5321, -0.5053, -0.5776]],
       requires_grad=True)


In [ ]:
# # Set hyperparameters
window_size = 4
embedding_size = 300
# More hyperparameters
learning_rate = 0.001
epochs = 50

In [ ]:
class sample_data():
    """Face Landmarks dataset."""

    def __init__(self,data):
      self.data = data
    def __len__(self):
       return len(self.data)        

    def __getitem__(self, idx):
    #   word_to_one_hot_list_for_tensor = []
    #   for each in self.data['text'][idx].split(' '):
    #     word_to_one_hot_list_for_tensor.append(word_to_one_hot(each,one_hot_vocabulary))
    #   # print("Print:",len([self.data['task_1'][idx]]))
        # def __getitem__(self, idx):
      # word_to_one_hot_list_for_tensor = []
      # for each in self.data['text'][idx].split(' '):
      #   word_to_one_hot_list_for_tensor.append(word_to_one_hot(each,one_hot_vocabulary))
      # print("Print:",len([self.data['task_1'][idx]]))
      return self.data['text'][idx], self.data['task_1'][idx]

def collate_fn(batch):
    word_to_one_hot_list_for_tensor = []
    for _data, _label in batch:
      for each in _data.split(' '):
        word_to_one_hot_list_for_tensor.append(word_to_one_hot(each,one_hot_vocabulary))
    return torch.Tensor(word_to_one_hot_list_for_tensor), torch.Tensor(_label)


      # return torch.Tensor(word_to_one_hot_list_for_tensor), torch.Tensor(self.data['task_1'][idx])

In [ ]:
sample_custom_dataset = sample_data(data)

In [ ]:
# DataLoader
def dataloader_for_model():
  # samle_custom_dataset = custom_dataset(data)
  # one_hot_tensor = torch.FloatTensor(full_one_hot_list)
  # context_tensor = torch.FloatTensor(binary_list)
  # # one_hot_tensor = torch.LongTensor(full_one_hot_list)
  # # context_tensor = torch.LongTensor(binary_list)
  # dataset = TensorDataset(samle_custom_dataset)

  loader = DataLoader(
      sample_custom_dataset,
      batch_size=32,
      num_workers=0,
      collate_fn = collate_fn,
      shuffle=True
  )
  return loader

In [ ]:
len(V)

In [ ]:
# Create model 

class Sentiment_Analysis(nn.Module):
  def __init__(self, length, embedding_size,model):
    super().__init__()
    # print("First.")
    self.input_layer = nn.Linear(in_features=length, out_features=20150, bias=False)
    # print(model.input_layer.weight.shape)
    self.fc1 = model.input_layer.weight
    self.fc2 = model.output.weight
    # print("Second.")
    self.pre_out = nn.Linear(in_features=20150, out_features=embedding_size, bias=False)
    
    self.final = nn.Linear(in_features=embedding_size, out_features=1, bias = False)


  def forward(self, one_hot):
    x=self.input_layer(one_hot)
    print(" x shape ",x.size())
    x = x.view(x.size(0),-1)
    # rel = nn.ReLU(x)
    x2 = torch.matmul(self.fc1, x)
    # print(x2.shape)
    x3 = torch.matmul(self.fc2, x2)
    # print(x3.shape)

    x4 = self.pre_out(x3)
    x5 = self.final(x4)
    # print(" x2 shape ",x2.size())
    # out = nn.LogSigmoid(x2)
    out = F.log_softmax(x5)
    return out

torch.cuda.empty_cache()
sentiment = Sentiment_Analysis(len(V), embedding_size,model)
torch.cuda.empty_cache()
if torch.cuda.is_available():
    sentiment.cuda()


In [ ]:
# Define optimizer and loss
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
optimizer = optim.Adam(sentiment.parameters(), lr=learning_rate)
# optimizer = optim.Adam(net.parameters(), lr=learning_rate)
torch.cuda.empty_cache()
# criterion = nn.CrossEntropyLoss()
criterion = nn.BCELoss()
# criterion = criterion.to(device)

In [ ]:
# # Define train procedure
# losses = []

# # load initial weights
# torch.cuda.empty_cache()
def train(dataset):
  print("Training started")
  # device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  total_loss = 0
  stopping_loss_threshold = 1.0
  early_stop = False
  for epoch in range(epochs):
    # print(len(dataset))
    counter = 0
    for data in dataset:
      counter += 1
      inputs, labels = data

      # inputs,labels=inputs.type(torch.int),labels.type(torch.int)
      inputs, labels = inputs.cuda(), labels.cuda()
      inputs, labels = inputs.to(device), labels.to(device)
      # labels = labels.unsqueeze(1)
      # print(labels.shape)
      # print(inputs.shape)



      sentiment.zero_grad()
      # inputs = torch.tensor(inputs).to(torch.long)
      log_probs = sentiment(inputs)
      # print(len(log_probs[0][0]))
      # loss = criterion(log_probs, torch.max(labels, 1)[1])
      loss = criterion(log_probs, labels)

      # loss = criterion(log_probs, labels)
      loss.backward()
      optimizer.step()

      total_loss += loss.item()
      if counter == 10:
        print('epoch:%d, batch:%d, loss: %.3f' %
          (epoch + 1, counter, total_loss / 10))
        total_loss = 0.0

      # if (total_loss / 100) <=stopping_loss_threshold:
      #   print("Early Stop.")
      #   print('epoch:%d, loss: %.3f' %
      #       (epoch + 1, total_loss / 100))
      #   early_stop = True
      #   break
      # else:
      # if i % 100 == 99:    # print every 100 mini-batches
      #   print('epoch:%d, batch:%d, loss: %.3f' %
      #     (epoch + 1, i + 1, total_loss / 100))
      #   # losses.append(total_loss/10)
      #   total_loss = 0.0
    # total_loss = 0.0

    # print(losses) 

    # if early_stop:
    #   print("Stopped")
    #   break
   
train(dataloader_for_model())

print("Training finished")



Training started
Yo
 x shape  torch.Size([800, 20150])


RuntimeError: ignored

Task 2B

In [ ]:
data_bangla = pd.read_csv('bengali_hatespeech.csv', sep=',')
# data_bangla = data.head(100)
bengali_stop_word_list = pd.read_csv('banglastopword.txt', sep='\s+', header=None)
bengali_stop_word_list = bengali_stop_word_list[0].tolist()
data_bangla['sentence'] = data_bangla['sentence'].str.replace('[{}]'.format(string.punctuation), '')
data_bangla['sentence'] = data_bangla['sentence'].str.replace('[{}]'.format('।'), '')
data_bangla['sentence'] = data_bangla['sentence'].str.lower()
data_bangla['sentence'] = data_bangla['sentence'].apply(lambda x: ' '.join([item for item in x.split() if item not in (bengali_stop_word_list)]))

In [ ]:
# Bengali Unique word list
bengali_V = list(data_bangla['sentence'].str.split(' ', expand=True).stack().unique())
print(len(bengali_V))
def word_to_one_hot(word, vocab):
  return vocab[word]

In [ ]:
bengali_all_word = all_word_corpus(data_bangla['sentence'])
len(bengali_all_word)